# Filtering Data 

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('apple').getOrCreate()

In [3]:
df = spark.read.csv("data/appl_stock.csv", header = True, inferSchema = True)

In [4]:
df.show(5)

+----------+----------+----------+------------------+------------------+---------+------------------+
|      Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+----------+----------+----------+------------------+------------------+---------+------------------+
only showing top 5 rows



In [5]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



* To filler out data use filter(). Similar to where clause in SQL. Returns Dataframe and can be combine with the select().

In [6]:
df.filter('Close < 500').show(3)

+----------+----------+----------+------------------+----------+---------+------------------+
|      Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+----------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|
|2010-01-06|214.379993|    215.23|        210.750004|210.969995|138040000|27.333178000000004|
+----------+----------+----------+------------------+----------+---------+------------------+
only showing top 3 rows



In [7]:
df.filter('Close < 500').select('Date').show(3)

+----------+
|      Date|
+----------+
|2010-01-04|
|2010-01-05|
|2010-01-06|
+----------+
only showing top 3 rows



* we can also use python statement in filter().

In [8]:
df.filter(df['Close'] < 500).show(3)

+----------+----------+----------+------------------+----------+---------+------------------+
|      Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+----------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|
|2010-01-06|214.379993|    215.23|        210.750004|210.969995|138040000|27.333178000000004|
+----------+----------+----------+------------------+----------+---------+------------------+
only showing top 3 rows



# Logical Operator

* Use & = and, | = or, ~ = not, == equality, where each condition should be specified in separate brackets ().

In [9]:
df.filter((df['Close'] < 500) & (df['Close'] > 400)).show(3)

+----------+------------------+----------+------------------+----------+---------+---------+
|      Date|              Open|      High|               Low|     Close|   Volume|Adj Close|
+----------+------------------+----------+------------------+----------+---------+---------+
|2011-07-26|        399.999996|404.499992|399.67998900000003|403.410007|119145600|52.265619|
|2011-09-16|395.54000099999996|400.500008|395.03000299999997|400.500008|174628300|51.888601|
|2011-09-19|        397.000008|413.229992|        395.199993|411.630009|205965200|53.330599|
+----------+------------------+----------+------------------+----------+---------+---------+
only showing top 3 rows



* In SQL

In [10]:
df.filter('Close < 500 and Close > 400').show(3)

+----------+------------------+----------+------------------+----------+---------+---------+
|      Date|              Open|      High|               Low|     Close|   Volume|Adj Close|
+----------+------------------+----------+------------------+----------+---------+---------+
|2011-07-26|        399.999996|404.499992|399.67998900000003|403.410007|119145600|52.265619|
|2011-09-16|395.54000099999996|400.500008|395.03000299999997|400.500008|174628300|51.888601|
|2011-09-19|        397.000008|413.229992|        395.199993|411.630009|205965200|53.330599|
+----------+------------------+----------+------------------+----------+---------+---------+
only showing top 3 rows



* To save the result as a list of row object use collect()

In [11]:
result = df.filter(df['Low'] == 197.16).collect()

In [12]:
result

[Row(Date='2010-01-22', Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)]

In [13]:
row = result[0]

* Save row as different Data Structure like dictionary

In [14]:
row.asDict()

{'Date': '2010-01-22',
 'Open': 206.78000600000001,
 'High': 207.499996,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.620401}

In [15]:
row.asDict()['Date']

'2010-01-22'

# GroupBy and Aggregate

Allow to group rows together based on some column value.

In [16]:
df = spark.read.csv('data/sales_info.csv', inferSchema=True, header = True)

In [17]:
df.show(5)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
+-------+-------+-----+
only showing top 5 rows



* The groupBy return group.GroupedData object - not a dataframe

In [18]:
df.groupBy('Company')

In [19]:
df.groupBy('Company').mean()

DataFrame[Company: string, avg(Sales): double]

In [20]:
df.groupBy('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [21]:
df.groupBy('Company').max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [22]:
df.groupBy('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [23]:
df.groupBy('Company').min().show()

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



* Aggeragate is perform for the whole dataFrame

In [24]:
df.agg({'Sales':'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



# Import functions

In [25]:
from pyspark.sql.functions import countDistinct, stddev, avg

In [26]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [27]:
df.select(countDistinct('Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [28]:
df.select(avg('Sales')).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [29]:
df.select(stddev('Sales')).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



* Aliasing

In [30]:
df.select(avg('Sales').alias('Average')).show()

+-----------------+
|          Average|
+-----------------+
|360.5833333333333|
+-----------------+



* formart float

In [31]:
from pyspark.sql.functions import format_number

In [32]:
df.select(avg('Sales').alias('Average')).select(format_number('Average', 2)).show()

+-------------------------+
|format_number(Average, 2)|
+-------------------------+
|                   360.58|
+-------------------------+



* Ordering and sorting

In [33]:
df.orderBy('Sales').show()  # default is accending order

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [34]:
df.orderBy(df['Sales'].desc()).show()   # descending order

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+

